In [11]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')

from util.dataframe_utils import analyse_columns
from util.datetime_utils import calculate_hour_sin_cos

I wanted to start toying around with the library, even though I haven't yet completed a comprehensive EDA. To keep things simple, I thought about generating a dataset that focuses primarily on the negative sentiments, since there are minimal missing values and I can start modelings things.

In [2]:
df = pd.read_csv('../data/unprocessed/Tweets.csv')

# the following columns were dropped entirely due to the large ratio of missing values

df = df.drop(['airline_sentiment_gold', 'negativereason_gold', 'tweet_coord'], axis=1)

In [3]:
# let's also rename the negative reason fields for consistency

df = df.rename(columns={'negativereason': 'negative_reason', 
                        'negativereason_confidence': 'negative_reason_confidence'})

Filtering the dataset by negative sentiments, dropping the original catergory, and verifying that there are no missing values in the negative_reasons field.

In [4]:
df = df[df['airline_sentiment'] == 'negative']
df = df.drop(['airline_sentiment'], axis=1)
df['negative_reason'].unique()

array(['Bad Flight', "Can't Tell", 'Late Flight',
       'Customer Service Issue', 'Flight Booking Problems',
       'Lost Luggage', 'Flight Attendant Complaints', 'Cancelled Flight',
       'Damaged Luggage', 'longlines'], dtype=object)

In [5]:
df.head()

,tweet_id,airline_sentiment_confidence,negative_reason,negative_reason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
3,570301031407624196,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,1.0000,Can't Tell,0.6842,Virgin America,jnardino,0,@VirginAmerica seriously would pay $30 a fligh...,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
15,570282469121007616,0.6842,Late Flight,0.3684,Virgin America,smartwatermelon,0,@VirginAmerica SFO-PDX schedule is still MIA.,2015-02-24 10:01:50 -0800,"palo alto, ca",Pacific Time (US & Canada)
17,570276917301137409,1.0000,Bad Flight,1.0000,Virgin America,heatherovieda,0,@VirginAmerica I flew from NYC to SFO last we...,2015-02-24 09:39:46 -0800,this place called NYC,Eastern Time (US & Canada)


In [6]:
analyse_columns(df)

,Column,Data Type,Missing Values,Missing Ratio (%),Unique Values
0,tweet_id,int64,0,0.00%,9089
1,airline_sentiment_confidence,float64,0,0.00%,719
2,negative_reason,object,0,0.00%,10
3,negative_reason_confidence,float64,0,0.00%,1409
4,airline,object,0,0.00%,6
5,name,object,0,0.00%,4973
6,retweet_count,int64,0,0.00%,12
7,text,object,0,0.00%,9087
8,tweet_created,object,0,0.00%,8991
9,tweet_location,object,3142,34.23%,1987


Again, just for simplicity, I want to get a working dataset, so I'm dropping the columns with a relatively high missing value count. 

In [7]:
df = df.drop(['tweet_location', 'user_timezone'], axis=1)
df.head()


,tweet_id,airline_sentiment_confidence,negative_reason,negative_reason_confidence,airline,name,retweet_count,text,tweet_created
3,570301031407624196,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800
4,570300817074462722,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800
5,570300767074181121,1.0000,Can't Tell,0.6842,Virgin America,jnardino,0,@VirginAmerica seriously would pay $30 a fligh...,2015-02-24 11:14:33 -0800
15,570282469121007616,0.6842,Late Flight,0.3684,Virgin America,smartwatermelon,0,@VirginAmerica SFO-PDX schedule is still MIA.,2015-02-24 10:01:50 -0800
17,570276917301137409,1.0000,Bad Flight,1.0000,Virgin America,heatherovieda,0,@VirginAmerica I flew from NYC to SFO last we...,2015-02-24 09:39:46 -0800


In [8]:
df['negative_reason'] = pd.Categorical(df['negative_reason']).codes
df['airline'] = pd.Categorical(df['airline']).codes

# Normalize numerical variables
df['airline_sentiment_confidence'] = (df['airline_sentiment_confidence'] - df['airline_sentiment_confidence'].mean()) / df['airline_sentiment_confidence'].std()
df['negative_reason_confidence'] = (df['negative_reason_confidence'] - df['negative_reason_confidence'].mean()) / df['negative_reason_confidence'].std()

# Convert 'tweet_created' to datetime
df['tweet_created'] = pd.to_datetime(df['tweet_created'])

# Extract features from 'tweet_created'
df['day'] = df['tweet_created'].dt.day
df['hour'] = df['tweet_created'].dt.hour
df['minute'] = df['tweet_created'].dt.minute
df['second'] = df['tweet_created'].dt.second

# Creating cyclical features for hour of day as an example
df['hour_sin'], df['hour_cos'] = zip(*df['hour'].apply(calculate_hour_sin_cos))

df[['hour', 'hour_sin', 'hour_cos']]

,hour,hour_sin,hour_cos
3,11,2.588190e-01,-0.965926
4,11,2.588190e-01,-0.965926
5,11,2.588190e-01,-0.965926
15,10,5.000000e-01,-0.866025
17,9,7.071068e-01,-0.707107
...,...,...,...
14631,12,1.224647e-16,-1.000000
14633,12,1.224647e-16,-1.000000
14634,12,1.224647e-16,-1.000000
14636,11,2.588190e-01,-0.965926


In [9]:
df['hour'].unique()

array([11, 10,  9,  8,  7,  5, 22, 21, 20, 17, 15, 14, 13, 12,  4,  3, 18,
       16,  2, 23,  6, 19,  1,  0], dtype=int32)

In [10]:
analyse_columns(df)

,Column,Data Type,Missing Values,Missing Ratio (%),Unique Values
0,tweet_id,int64,0,0.00%,9089
1,airline_sentiment_confidence,float64,0,0.00%,719
2,negative_reason,int8,0,0.00%,10
3,negative_reason_confidence,float64,0,0.00%,1409
4,airline,int8,0,0.00%,6
5,name,object,0,0.00%,4973
6,retweet_count,int64,0,0.00%,12
7,text,object,0,0.00%,9087
8,tweet_created,"datetime64[ns, UTC-08:00]",0,0.00%,8991
9,hour_of_day,int32,0,0.00%,24
